In [1]:
import requests

In [2]:
URL = 'https://yozm.wishket.com/magazine/list/develop/?page=6'

In [3]:
res = requests.get(URL, headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
}) # User-Agent : python/requests

In [ ]:
res.status_code, res.text #, res.json()

In [7]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text)

In [ ]:
soup.find_all('a', class_="item-title")
soup.find_all('img', class_='thumbnail-image')

In [ ]:
boxes = soup.select('div.list-item-link') # 여러 개 # 게시글들 박스
# selector
# class -> .
# id -> #
DOMAIN_URL = 'https://yozm.wishket.com'
results = []
for box in boxes:
    title_tag = box.select_one('.item-title') # 한 개 # 게시글 제목 or 링크, 내용
    # print(title_tag.text.strip()) # 제목
    # print(DOMAIN_URL + title_tag.attrs['href']) # 게시글 링크
    results.append({
        '제목': title_tag.text.strip(),
        '링크': DOMAIN_URL + title_tag.attrs['href'],
        # 작성일시, 작성자, ...
    })
results

실습 1
- HTML 문서 내에 본문을 나타내는 태그의 ID를 찾아 태그내의 내용을 출력해주세요.
- https://ko.wikipedia.org/wiki/위키백과

In [21]:
res = requests.get('https://ko.wikipedia.org/wiki/위키백과')
res.status_code

200

In [ ]:
soup = BeautifulSoup(res.text)
# id = mw-content-text
content_tag = soup.select_one('#mw-content-text')
print(content_tag.text.strip())

실습 2
- HTML 문서 내에서 공고문 하나하나를 구분하는 클래스를 찾아 해당 태그내의 내용을 출력해주세요
- https://www.saramin.co.kr/zf_user/jobs/public/list

In [28]:
res = requests.get('https://www.saramin.co.kr/zf_user/jobs/public/list', headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
})

res.status_code

200

In [32]:
res.text.find('완주우체국')

2086533

In [33]:
soup = BeautifulSoup(res.text)

In [ ]:
boxes = soup.select('.list_item')

for box in boxes:
    print(box.text)

In [ ]:
type(boxes)
for box_tag in boxes:
    ...

In [ ]:
type(boxes[0])

In [ ]:
type(content_tag)

실습 3
- 사이트에서 onclick 속성에 goView가 포함된 태그를 하나만 선택해 출력해주세요.
- https://youth.seoul.go.kr/infoData/sprtInfo/list.do


In [48]:
res = requests.get('https://youth.seoul.go.kr/infoData/sprtInfo/list.do')
res.status_code

200

In [50]:
res.text.find('goView')

114193

In [53]:
soup = BeautifulSoup(res.text)
soup.select_one('[onclick*="goView"]').text

'서울시 &lt;미래 청년 일자리 2차&gt; 신청 안내 .....1881026'

In [58]:
len(soup.select('[class~="view-list"]'))

1

실습 4
- 사이트에서 주요뉴스 내용과 일자별 뉴스들을 리스트로 정리해주세요
- https://ko.wikipedia.org/wiki/포털:요즘_화제
- 결과 : [‘주요 뉴스’, '8월 25일, 파키스탄에서 일어난 홍수로 1,000명 이상의 주민이 사망하고, 가축 70,000마리 이상이 죽었다.’, ...,
'Current events of 2022년 8월 29일\xa0(2022-08-29) (월요일)', ‘대한민국의 0시 기준 누적 \u200b확진자 수가 23,026,960명으로
집계되었다. 전날 0시 대비 43,142명(국내 42,782, 해외유입 360)이 늘었다.’, ...]

In [59]:
res = requests.get('https://ko.wikipedia.org/wiki/포털:요즘_화제')
res.status_code

200

In [60]:
soup = BeautifulSoup(res.text)

In [61]:
vevents = soup.select('.vevent')[:-3]

In [69]:
results = []

In [70]:
# 주요뉴스 처리`
vevent_tag = vevents[0]
tr_tags = vevent_tag.select('tbody table tr')
results.append(tr_tags[0].text.strip()) # 제목
li_tags = tr_tags[1].select('li')
# li_tags[0].text
results.extend([li_tag.text for li_tag in li_tags]) # 내용
results

['주요 뉴스',
 '8월 5일, 방글라데시의 총리 셰이크 하시나(사진)가 반정부 항의에 이어 사임을 하고 인도로 달아났다.',
 '7월 31일, 하마스의 지도자 이스마일 하니예가 이란 테헤란에서 암살되었다.',
 '7월 26일, 2024년 하계 올림픽이 프랑스 파리에서 개막했다.',
 '7월 22일, 에티오피아에서 발생한 두 차례의 산사태로 257명 이상이 사망했다.',
 '7월 21일, 조 바이든 미국 대통령이 2024년 미국 대통령 선거에 불출마하겠다고 밝혔다.']

In [72]:
for vevent_tag in vevents[1:]: # 주요뉴스 뒷부분 (일자별 뉴스 정리)
    # 날짜
    tr_tags = vevent_tag.select('tbody table tr')
    results.append(tr_tags[0].text.strip()) # 제목
    # 뉴스들
    tr_tags = vevent_tag.select('tbody tr') # 2개 -> 3개
    # print(len(tr_tags))
    tr_tag = tr_tags[-1] # 뉴스 기사들이 담긴 태그
    li_tags = tr_tag.select('li')
    results.extend([li_tag.text for li_tag in li_tags])

In [73]:
res = requests.get('http://www.cine21.com/rank/boxoffice/domestic')

In [74]:
res.text.find('파일럿')

-1

In [96]:
res = requests.post('http://www.cine21.com/rank/boxoffice/domestic_content', data={
    'page': 4,
    'genre': 'all',
    'period': '2024-07-31'
})
res.status_code

200

In [97]:
res.text.find('파일럿')

-1

In [98]:
soup = BeautifulSoup(res.text) # html

In [ ]:
movies = soup.select('.boxoffice_li')

results = []

for movie_tag in movies:
    title = movie_tag.select_one('.mov_name').text # 제목
    temp = {
        '제목': title
    }
    # print(title)
    li_tags = movie_tag.select('li')
    # print(li_tags)
    for li_tag in li_tags:
        tit_text = li_tag.select_one('.tit').text
        value_text = li_tag.text.replace(tit_text, '')
        # print(tit_text, value_text, sep="\t\t\t")
        temp[tit_text] = value_text

    results.append(temp)

results

In [100]:
results = [] #
for page in range(1, 11):
    res = requests.post('http://www.cine21.com/rank/boxoffice/domestic_content', data={
        'page': page,
        'genre': 'all',
        'period': '2024-07-31'
    })
    soup = BeautifulSoup(res.text) # html
    movies = soup.select('.boxoffice_li')
    # results = [] #
    for movie_tag in movies:
        title = movie_tag.select_one('.mov_name').text # 제목
        temp = { '제목': title }
        li_tags = movie_tag.select('li')
        for li_tag in li_tags:
            tit_text = li_tag.select_one('.tit').text
            value_text = li_tag.text.replace(tit_text, '')
            temp[tit_text] = value_text

        results.append(temp)


In [101]:
len(results)

200

In [102]:
res = requests.get('https://jsonplaceholder.typicode.com/posts')
res.status_code

200

In [107]:
type(res.text)

str

In [109]:
# print(res.text)
import json

datas = json.loads(res.text)
type(datas)

list

In [ ]:
# json

In [110]:
datas[0]
# [
#  {

#   html str
# <li>

{'userId': 1,
 'id': 1,
 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit',
 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}

실습 1
- 사이트 내 공지사항을 하나씩 출력해주세요.
- https://didimteo.startup-plus.kr/default.do


실습 2
- 실습 1의 결과에서 제목과 날짜를 분리해 딕셔너리 형태로 저장해주세요
- 결과 : [{'제목': '[채용] 광운대학교 산학협력단(서울창업디딤터) 직원 채용 공고(~6/17)', '날짜': '2022.06.10'},
{'제목': '[서울창업디딤터] 대강의실 및 공동작업실 이용 신청 방식 변경', '날짜': '2022.06.04’}, ... ]

실전 1 - 네이버 웹툰 크롤링
- 네이버 웹툰들을 다음과 같이 모두 크롤링해주세요.
- https://comic.naver.com/webtoon
- 결과 : [{'제목': '참교육', '링크': 'https://comic.naver.com/webtoon/list?titleId=758037&weekday=mon’, ‘요일’: ’월’}, {'제
목': '신의 탑', '링크': 'https://comic.naver.com/webtoon/list?titleId=183559&weekday=mon’ , ‘요일’: ’월’}, ... ]

In [111]:
res = requests.get('https://comic.naver.com/api/webtoon/titlelist/weekday?order=user')
res.status_code

200

In [113]:
import json
datas = json.loads(res.text)

In [ ]:
for dayofweek, day_webtoons in datas['titleListMap'].items():
    print(dayofweek)
    print(len(day_webtoons))
    for webtoon in day_webtoons:
        print(webtoon)
        break

In [124]:
w_e2k = dict(zip(datas['titleListMap'].keys(), ['목', '수', '토', '일', '금', '월', '화']))

In [125]:
w_e2k

{'THURSDAY': '목',
 'WEDNESDAY': '수',
 'SATURDAY': '토',
 'SUNDAY': '일',
 'FRIDAY': '금',
 'MONDAY': '월',
 'TUESDAY': '화'}

In [126]:
datas['titleListMap']['MONDAY']

WEBTOON_DETAIL_URL = 'https://comic.naver.com/webtoon/list?titleId={}&tab={}'
webtoons = [{
        '제목': webtoon['titleName'],
        # https://comic.naver.com/webtoon/list?titleId=769209&tab=wed
        '링크': WEBTOON_DETAIL_URL.format(webtoon['titleId'], dayofweek[:3].lower()),
        '요일': w_e2k[dayofweek],
    } for dayofweek, day_webtoons in datas['titleListMap'].items() for webtoon in day_webtoons]

In [ ]:
webtoons

In [141]:
res = requests.get('https://www.rocketpunch.com/api/jobs/template?page=2')
res.status_code

200

In [142]:
res.text.find('스프린트')

-1

In [143]:
import json
datas = json.loads(res.text)

In [144]:
res.text[:100]

'{"data":{"template":"<div class=\\"ui job items segment company-list\\"><div class=\\"ui blank filter s'

In [145]:
datas.keys()

dict_keys(['data', 'meta'])

In [146]:
soup = BeautifulSoup(datas['data']['template'])

In [ ]:
companies = soup.select('.company')

results = []
for company_tag in companies:
    name = company_tag.select_one('.company-name').text.strip()
    temp = []
    for job_detail_tag in company_tag.select('.job-detail'):
        temp.append({
            '공고명': job_detail_tag.select_one('.job-name').text.strip()
        })
    results.append({
        '기업명': name,
        '채용공고들': temp
    })
results